<a href="https://colab.research.google.com/github/ArthurBiragnet/Dolphin/blob/optimize/Dolphin_Percent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Library
import requests
import json
from datetime import datetime
import scipy.optimize as optimize
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 100000)
pd.set_option("display.precision", 12)
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_data(endpointApi, date=None, full_response=False, columns=list()):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    payload = {'date': date, 'fullResponse': full_response}
    res = requests.get(URL + endpointApi,
                      params=payload,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [3]:
def put_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.put(url=URL + endpointApi,
                      data=json.dumps(content),
                      auth=AUTH,
                      headers = {"content-type": "application/json"},
                      verify=False)
    return res.content.decode('utf-8')

In [4]:
def put_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.put(url=URL + endpointApi,
                      data=json.dumps(content),
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [5]:
def post_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    #json.dumps(content),
    res = requests.post(url=URL + endpointApi,
                      data=content,
                      auth=AUTH,
                      headers = {"content-type": "application/json"},
                      verify=False)
    return res.content.decode('utf-8')

In [6]:
def convert_currency(value, df_currency_rate):
    value, currency_name = value.split(' ')
    rate = currencyRateToEUR(currency_name, df_currency_rate)
    
    new_value = float(value.replace(',', '.')) * rate
    return new_value

def convert_datetime(value):
    return datetime.strptime(value, '%Y-%m-%d').date()

def convert_float(value):
    return float(value.replace(',', '.'))

def convert_int(value):
    return int(value)

def convert_boolean(value):
    return json.loads(value)

def df_convert_type(df, df_currency_rate):
        for col in df.columns:
            convert_values = []
            for elt in df[col]:
                if elt is np.nan:
                    convert_values.append(np.nan)
                    continue

                elt_type = elt['type']
                elt_value = elt['value']
                if elt_type == 'currency_value':
                    elt_value = convert_currency(elt_value, df_currency_rate)
                elif elt_type == 'date':
                    elt_value = convert_datetime(elt_value)
                elif elt_type in ['double', 'percent']:
                    elt_value = convert_float(elt_value)
                elif elt_type in ['asset', 'int32']:
                    elt_value = convert_int(elt_value)
                elif elt_type == 'boolean':
                    elt_value = convert_boolean(elt_value)
                elif elt_type not in ['asset_type', 'string', 'asset_currency', 'date-time',
                                     'asset_sub_type', 'asset_status', 'asset_quote_type',
                                     'liquidity_algorithm', 'portfolio_lock_mode', 'portfolio_type']:
                    print(elt)
                convert_values.append(elt_value)
            df[col] = convert_values
        return df

In [7]:
def getCurrencyRate():
    d = []
    currency_json = json.loads(get_data('currency'))
    for currency in currency_json:
        currency_id = currency.get('id')
        rate = get_data('currency/rate/{}/to/EUR'.format(currency_id))
        if len(rate) == 0:
            continue
        rate_value = json.loads(rate)['rate']['value']
        d.append([currency_id, convert_float(rate_value)])
    return pd.DataFrame(d, columns=['currency src', 'rate'])
    
df_currency_rate = getCurrencyRate()
df_currency_rate

,currency src,rate
0,USD,0.851208716377
1,EUR,1.000000000000
2,JPY,0.008066467694
3,BEF,0.024789352477
4,CYP,1.708601441376
5,EEK,0.063911648537
6,MTL,2.329373398556
7,SKK,0.033193918874
8,USN,0.851208716377
9,USS,0.851208716377


In [8]:
def currencyRateToEUR(cur_src, df_currency_rate):
    return df_currency_rate[df_currency_rate['currency src'] == cur_src]['rate'].values[0]

#currencyRateToEUR('USD', df_currency_rate)

In [9]:
"""data = get_data('asset/')
df = pd.read_json(data)
df_assets = df_convert_type(df, df_currency_rate)
df_assets"""

"data = get_data('asset/')\ndf = pd.read_json(data)\ndf_assets = df_convert_type(df, df_currency_rate)\ndf_assets"

In [10]:
def get_df_asset(asset_date, df_currency_rate, asset_id='ASSET_DATABASE_ID', asset_label='LABEL',
                 asset_type='TYPE', asset_last_close_value='LAST_CLOSE_VALUE_IN_CURR', asset_min_buy_amount='MIN_BUY_AMOUNT'):
    data = get_data('asset?columns={}&columns={}&columns={}&columns={}&columns={}&columns={}&columns={}&date={}&columns=CREATION_DATE'
                   .format(asset_id, asset_label, asset_type, asset_last_close_value, asset_min_buy_amount, 'nav', 'asset_fund_info_decimalisation', asset_date))
    df_asset = pd.read_json(data)
    df_asset = df_convert_type(df_asset, df_currency_rate)
    df_asset = df_asset[(df_asset['LAST_CLOSE_VALUE_IN_CURR'].notna()) | (df_asset['TYPE'] == 'PORTFOLIO')].reset_index() #FIXME es assets avec LAST_CLOSE_VALUE_IN_CURR = NAN bug lors de l'ajout dans un portefeuille
    return df_asset

#df_ETF = get_df_asset('2016-06-01', df_currency_rate)
#df_ETF

In [11]:
def get_cotation_values(asset_id, start_date, end_date, df_currency_rate):
    data = get_data('asset/{}/quote?start_date={}&end_date={}'
                    .format(asset_id, start_date, end_date))
    df_cotation = pd.read_json(data)
    df_cotation = df_convert_type(df_cotation, df_currency_rate)
    return df_cotation

#df_cotation = get_cotation_values('1845', '2016-06-01', '2020-09-30', df_currency_rate)
#df_cotation

In [12]:
def get_ratio(df_currency_rate):
    data = get_data('ratio')
    df_ratio = pd.read_json(data)
    return df_ratio

df_ratio = get_ratio(df_currency_rate)
df_ratio

,id,type,name,is_benchmark_needed,is_percent
0,11,Ratio,Correlation,True,False
1,7,Ratio,Exposition action,False,True
2,15,Ratio,Exposition action,False,True
3,8,Ratio,RDT,False,True
4,13,Ratio,Rendement,False,True
5,9,Ratio,Rendement_annualise,False,True
6,12,Ratio,Sharpe,False,False
7,14,Ratio,VaR historique,False,True
8,10,Ratio,Volatilite,False,True


In [13]:
def post_ratio(ratios, assets_id, start_date, end_date, bench=None, frequency=None):
    payload = "{{'ratio'={},'asset'={},'start_date'={},'end_date'={}, 'bench'=null, 'frequency':null}}".format(ratios, assets_id, start_date, end_date)
    reponse = post_data('ratio/invoke', payload)
    return reponse
    
#get Rendement_annualise, Sharpe, Volatilite, Rendement de l'asset 1845 et 1991 entre '2016-06-01' et '2020-09-30'
#post_ratio([9, 12, 10, 13], [1845], '2016-06-01', '2020-09-30')

In [14]:
def get_df_ratios(assets_id, begin_date, end_date):
    json_ratios = json.loads(post_ratio([10, 12, 13], assets_id, begin_date, end_date))
    ratios = []
    for asset_id in assets_id:
        str_asset_id = str(asset_id)
        volatilite = convert_float(json_ratios[str_asset_id]['10']['value'])
        sharpe = convert_float(json_ratios[str_asset_id]['12']['value'])
        rendement = convert_float(json_ratios[str_asset_id]['13']['value'])
        ratios.append([asset_id, volatilite, rendement, sharpe])
    return pd.DataFrame(ratios, columns=['ASSET_DATABASE_ID', 'volatilite', 'rendement', 'sharpe'])

#get_df_ratios([1845, 1991], '2016-06-01', '2020-09-30')

In [15]:
def get_portfolio_id(df_assets, label):
    portfolio_id = df_assets.loc[(df_assets['TYPE'] == 'PORTFOLIO') & (df_assets['LABEL'] == label)]['REST_OBJECT_ID'].values[0]
    return int(portfolio_id)

In [16]:
def get_portfolio(portfolio_id):
    data = get_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id))
    df_portfolio = pd.read_json(data)
    return df_portfolio

In [17]:
def get_assets_portfolio(df_portfolio, date):
    if date not in df_portfolio['values']:
        return np.NaN
    return df_portfolio['values'][date]

In [18]:
def portfolio_json_assets(assets):
    json = ''
    for asset in assets:
        json += '{{"asset":{{"asset": {}, "quantity": {}}}}},'.format(asset["asset"]["asset"], asset["asset"]["quantity"])
    return json[:-1]
    
def seralize_portfolio_content(df_portfoliot, assets=None):
    label = df_portfoliot['label'][0]#"EPITA_PTF_10"
    currency = df_portfoliot['currency'][0]#"EUR"
    type_ = df_portfoliot['type'][0]
    date = '2016-06-01'

    if assets is None:
      res = '{{"label": "{}", "currency": {{"code": "{}"}}, "type": "{}", "values": {{}} }}'.format(label, currency, type_, date)

    else:
      assets = portfolio_json_assets(assets)
      res = '{{"label": "{}", "currency": {{"code": "{}"}}, "type": "{}", "values": {{"{}": [{}]}}}}'.format(label, currency, type_, date, assets)
    return json.loads(res)

def deseralize_portfolio_content(json_portfolio):
    json_portfolio = pd.json_normalize(json_portfolio)
    
    cols = [col for col in json_portfolio.columns if col.split('.')[0] == 'values']
    json_portfolio = json_portfolio[cols]
    json_portfolio.columns = json_portfolio.columns.map(lambda x: x.split(".")[0])

    json_assets = json_portfolio['values'][0]
    assets = []
    for asset in json_assets:
        assets.append(asset)
    return assets

In [19]:
def put_portfolio(portfolio_id, df_portfolio, assets):
    content = seralize_portfolio_content(df_portfolio, assets)
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), content)

In [20]:
def reset_assets_portfolio(portfolio_id, df_portfolio):
    content = seralize_portfolio_content(df_portfolio)
    content['values'] = {}
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), content)

In [21]:
import matplotlib.pyplot as plt
import random
#FIXME il faudrait imposer les contraintes du sujet aux portfolios générés (ex: nav 1 et 15%, stock > 50%, ...)
def get_Markowitz_Bullet(begin_date, end_date, best_assets=None, assets=None, iterations=10000):
    if assets is None:
        assets = get_df_asset(begin_date, df_currency_rate)
        assets = assets[assets['TYPE'] != 'PORTFOLIO']
    assets_id = [int(asset_id) for asset_id in assets['ASSET_DATABASE_ID'].values]
    #assets_id = random.choices(assets_id, k=5)
    
    #dataframe de tous les ratio 12 Sharpe 13 Rendement 10 Volatile pour chaque assets
    df_ratios = get_df_ratios(assets_id, begin_date, end_date)
    assets = assets.merge(df_ratios, on='ASSET_DATABASE_ID')
    
    portfolio_returns = []
    portfolio_volatilities = []
    portfolio_sharpes = []
    
    returns = assets['rendement'].values
    volatilities = assets['volatilite'].values
    sharpes = assets['sharpe'].values
    
    for i in range(iterations):
        weights = np.random.dirichlet(np.ones(assets.shape[0]), size=1)
        weights = weights[0]

        total_returns = np.dot(returns, weights)
        total_volatilities = np.dot(volatilities, weights)
        total_sharpes = np.dot(sharpes, weights)
        
        #rates_returns = (1 + returns)**(365/1582) - 1

        #portfolio_sharpes.append((np.sum(rates_returns) - 0.05)/np.sum(total_volatilities))
        #print((np.sum(taux_returns) - 0.05)/np.sum(total_volatilities))
        #print(-(taux_returns - 0.05)/np.dot(volatilities, weights))
        
        portfolio_returns.append(total_returns)
        portfolio_volatilities.append(total_volatilities)
        portfolio_sharpes.append(total_sharpes)
        
    portfolio_returns = np.array(portfolio_returns)
    portfolio_volatilities = np.array(portfolio_volatilities)
    portfolio_sharpes = np.array(portfolio_sharpes)
    
    plt.figure(figsize=(18,15))
    plt.scatter(portfolio_volatilities, portfolio_returns, c = portfolio_returns/portfolio_volatilities)
                #c=(portfolio_sharpes)) #FIXME
    plt.xlabel('Volatility')
    plt.ylabel('Return')
    plt.colorbar(label='Sharpe ratio')
    plt.title('Markowitz Bullet (voile de bateau)')
    
    if best_assets is not None:
        returns = best_assets['rendement'].values
        volatilities = best_assets['volatilite'].values
        sharpes = best_assets['sharpe'].values
        best_weights = best_assets['weights'].values

        best_portfolio_returns = np.dot(returns, best_weights)
        best_portfolio_volatilies = np.dot(volatilities, best_weights)
        best_sharpe = best_portfolio_returns / best_portfolio_volatilies
        plt.scatter(best_portfolio_volatilies, best_portfolio_returns, c='red', marker='+', s=[500])

        print('Best portfolio have a sharpe ratio = {}'.format(best_sharpe))

#get_Markowitz_Bullet('2016-06-01', '2020-09-30')

In [22]:
def portfolio_stats(assets, weights):
    returns = assets['rendement'].values
    volatilities = assets['volatilite'].values
    sharpes = assets['sharpe'].values

    total_returns = np.dot(returns, weights) #FIXME
    total_volatilities = np.dot(volatilities, weights) #FIXME

    #rates_returns = (1 + returns)**(365/1582) - 1
    #portfolio_sharpes.append((np.sum(rates_returns) - 0.05)/np.sum(total_volatilities))

    #total_sharpes = np.dot(weights, sharpes)
    total_sharpes = total_returns / total_volatilities #FIXME
    
    return {'return': total_returns, 'volatility': total_volatilities, 'sharpe': total_sharpes}

In [23]:
def minimize_neg_sharpe(weights, assets):#on veut maximiser sharpe
    return -portfolio_stats(assets, weights)['sharpe']

In [24]:
"""def nav_max_constraint(weights): #FIXME ON doit check > 1 %
     assets = assets_global
     index_max = np.argmax(weights)
     asset_nav_max = assets['LAST_CLOSE_VALUE_IN_CURR'].values[index_max] #FIXME change close to nav
     weight_max = max(weights)
     a = weight_max * asset_nav_max - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.095 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_nav_max
     res_nav_max = a
     return a

def nav_min_constraint(weights):
     assets = assets_global
     index_min = np.argmin(weights)
     asset_nav_min = assets['LAST_CLOSE_VALUE_IN_CURR'].values[index_min] #FIXME change close to nav
     weight_min = min(weights)
     a = weight_min * asset_nav_min - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.015 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_nav_min
     res_nav_min = a
     return a"""

def nav_max_constraint(weights): #FIXME ON doit check > 1 %
     assets = assets_global
     nb_weights = len(weights)

     max_nav = None
     for i in range(nb_weights):
       nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
       if max_nav is None:
         max_nav = nav
       else:
         max_nav = max(max_nav, nav)
     res = max_nav - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.095 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_nav_max
     res_nav_max = res
     return res

def nav_min_constraint(weights):
     assets = assets_global

     nb_weights = len(weights)

     min_nav = None
     for i in range(nb_weights):
       nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
       if min_nav is None:
         min_nav = nav
       else:
         min_nav = min(min_nav, nav)
     res = min_nav - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.015 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_nav_min
     res_nav_min = res
     return res

def stock_constraint(weights):
     assets = assets_global

     stock_assets_id = [asset_id for asset_id in assets[assets['TYPE'] == 'STOCK'].index]
     stock_weights = weights[stock_assets_id]
     stock_nav = assets['LAST_CLOSE_VALUE_IN_CURR'].values[stock_assets_id] #FIXME close to nav

     res = np.dot(stock_weights, stock_nav) - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * stock_percent_global #FIXME augmenter au-dessus de 0.5 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

     global res_assets_number
     res_assets_number = res
     return res

def optimize_sharpe(assets, stock_percent):
    nb_assets = assets.shape[0]

    global assets_global
    assets_global = assets

    global stock_percent_global
    stock_percent_global = stock_percent
    
    constraints = ({'type': 'eq', 'fun': nav_min_constraint},
                   {'type': 'eq', 'fun': nav_max_constraint},
                   {'type': 'eq', 'fun': stock_constraint})
    bounds = tuple((1, None) for x in range(nb_assets))
    initializer = nb_assets * [1/ nb_assets]
    
    best_sharpe = optimize.minimize(minimize_neg_sharpe, initializer, args = (assets), method='SLSQP', bounds=bounds, constraints=constraints)
    return best_sharpe.x

In [25]:
def check_nav_max_constraint(assets):
    weights = assets['weights'].values
    nb_weights = len(weights)

    max_nav = None
    for i in range(nb_weights):
      nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
      if max_nav is None:
        max_nav = nav
      else:
        max_nav = max(max_nav, nav)
    return max_nav <= np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.10
     
def check_nav_min_constraint(assets):
    weights = assets['weights'].values
    nb_weights = len(weights)

    min_nav = None
    for i in range(nb_weights):
      nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
      if min_nav is None:
        min_nav = nav
      else:
        min_nav = min(min_nav, nav)
    return min_nav >= np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.01 #FIXME réduire à en-dessous de 0.10 pour être large (il va converger vers 0 mais il sera jamais égale à 0)

def check_nav_constraint(assets):
     check_max = check_nav_max_constraint(assets)
     check_min = check_nav_min_constraint(assets)

     #print('check_max = {}\ncheck_min = {}'.format(check_max, check_min))
     return check_max and check_min

def check_stock_constraint(assets):
     weights = assets['weights'].values

     stock_assets_index = [asset_index for asset_index in assets[assets['TYPE'] == 'STOCK'].index]
     stock_weights = weights[stock_assets_index]
     stock_nav = assets['LAST_CLOSE_VALUE_IN_CURR'].values[stock_assets_index] #FIXME close to nav

     return np.dot(stock_weights, stock_nav) >= np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.5

def check_quantity_constraint(assets):
    for _, row in assets.iterrows():
      if ((row['MIN_BUY_AMOUNT'] is not np.NaN) and (row['weights'] < row['MIN_BUY_AMOUNT'])):
          return False
    return True

def check_decimalisation_constraint(assets):
    return assets[assets['asset_fund_info_decimalisation'].notna()].empty

In [26]:
import scipy as sp
import scipy.optimize as scopt
import scipy.stats as spstats

def select_best_assets(begin_date, end_date, df_currency_rate, stock_percent=0.6):
    assets = get_df_asset(begin_date, df_currency_rate)
    #remove portfolios from assets
    assets = assets[assets['TYPE'] != 'PORTFOLIO']
    assets_id = [int(asset_id) for asset_id in assets['ASSET_DATABASE_ID'].values]
    
    #dataframe de tous les ratio 12 Sharpe 13 Rendement 10 Volatile pour chaque assets
    df_ratios = get_df_ratios(assets_id, begin_date, end_date)
    assets = assets.merge(df_ratios, on='ASSET_DATABASE_ID')

    #select assets with best sharpe : #FIXME Kmeans pour la corrélation
    best_assets = assets.sort_values(by=['sharpe'], ascending=False)[:20].reset_index().drop(columns=['level_0', 'index'])#FIXME pas 20
    best_assets = best_assets[best_assets['MIN_BUY_AMOUNT'].isna()]

    global res_nav_max
    res_nav_max = 0
    global res_nav_min
    res_nav_min = 0
    global res_assets_number
    res_assets_number = 0

    weights = optimize_sharpe(best_assets, stock_percent)


    print('weights : {}'.format(np.sort(weights)))
    print('nav score max = {}'.format(res_nav_max))
    print('nav score min = {}'.format(res_nav_min))
    print('assets stock score = {}'.format(res_assets_number))

    best_assets['weights'] = (weights * 100).astype(int)
    
    print('check nav constraint : {}'.format(check_nav_constraint(best_assets)))
    print('check stock constraint : {}'.format(check_stock_constraint(best_assets)))
    print('check minimal buy amount constraint : {}'.format(check_quantity_constraint(best_assets)))
    print('check declimalisation constraint : {}'.format(check_decimalisation_constraint(best_assets)))

    return best_assets

In [27]:
def post_assets_selected_in_portfolio(assets, begin_date, df_currency_rate):
    assets_id_quantity = [{"asset": {"asset": asset['ASSET_DATABASE_ID'], "quantity": asset['weights']}} for _, asset in assets.iterrows()]
    df_assets = df_convert_type(pd.read_json(get_data('asset/')), df_currency_rate)
    
    portfolio_id = get_portfolio_id(df_assets, 'EPITA_PTF_10')
    df_portfolio = get_portfolio(portfolio_id)
    
    reset_assets_portfolio(portfolio_id, df_portfolio)#on reset le contenu du portfolio (plus safe)
    
    df_portfolio = get_portfolio(portfolio_id)
    put_portfolio(portfolio_id, df_portfolio, assets_id_quantity)

In [28]:
def compare_portfolios(our_portfolio_id, ref_portfolio_id, begin_date, end_date):    
    ratios = json.loads(post_ratio([12], [our_portfolio_id, ref_portfolio_id], begin_date, end_date))
    print('sharpe ratio in our portfolio : {}\nsharpe ratio in ref portfolio : {}'
          .format(ratios[str(our_portfolio_id)]['12']['value'], ratios[str(ref_portfolio_id)]['12']['value']))
    return ratios[str(our_portfolio_id)]['12']['value']

In [29]:
def main():
  begin_date = '2016-06-01'
  end_date = '2020-09-30'
  #df_currency_rate = getCurrencyRate()#FIXME

  assets_selected = select_best_assets(begin_date, end_date, df_currency_rate, 0.69)
  #get_Markowitz_Bullet('2016-06-01', '2020-09-30', best_assets=assets_selected, iterations=100000)
  post_assets_selected_in_portfolio(assets_selected, begin_date, df_currency_rate)

  df_assets = df_convert_type(pd.read_json(get_data('asset/')), df_currency_rate)

  our_portfolio_id = get_portfolio_id(df_assets, 'EPITA_PTF_10')
  our_portfolio = get_portfolio(our_portfolio_id)

  reset_assets_portfolio(our_portfolio_id, our_portfolio)#FIXME ATTENTIN A SUPRIMMER AVANT DE POST POUR AVOIR LES ACTIFS DANS LE PORTFOLIO
  
  ref_portfolio_id = get_portfolio_id(df_assets, 'REF')
  compare_portfolios(our_portfolio_id, ref_portfolio_id, begin_date, end_date)

main()

weights : [4.74805962e+00 8.62337519e+00 4.34775729e+01 1.52946204e+02
 1.76462310e+02 3.33714664e+02 2.10799900e+03 2.35760470e+03
 2.38203881e+03 2.64564481e+03 3.30612948e+03 3.48172924e+03
 3.58504261e+03 5.71103976e+03 7.37393981e+03 1.75381179e+04
 2.29785982e+04 2.51409421e+04 4.80952324e+05]
nav score max = 621.5925179508049
nav score min = -0.0033080064313253388
assets stock score = -0.2111807856708765
check nav constraint : True
check stock constraint : True
check minimal buy amount constraint : True
check declimalisation constraint : True
sharpe ratio in our portfolio : 2,678087286221
sharpe ratio in ref portfolio : -0,337925081462


In [30]:
def main():
  begin_date = '2016-06-01'
  end_date = '2020-09-30'
  """stock_percent = [0.55, 0.56, 0.57, 0.58, 0.59,
                   0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69,
                   0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79,
                   0.8, 0.85, 0.9, 0.95]"""
  stock_percent = [0.55, 0.56, 0.58, 0.59, 0.6, 0.69, 0.72, 0.75, 0.8, 0.85, 0.9, 0.95]
  #df_currency_rate = getCurrencyRate()#FIXME
  max_sharpe = (stock_percent[0], stock_percent[0])
  for i in range(len(stock_percent)):
    print("-------------------------------------------------------------------------------------------------------")
    print("STOCK PERCENT = " + str(stock_percent[i]))
    assets_selected = select_best_assets(begin_date, end_date, df_currency_rate, stock_percent[i])
    if (check_nav_constraint(assets_selected) and check_stock_constraint(assets_selected) and
        check_quantity_constraint(assets_selected) and check_decimalisation_constraint(assets_selected)):
      #get_Markowitz_Bullet('2016-06-01', '2020-09-30', best_assets=assets_selected, iterations=100000)
      post_assets_selected_in_portfolio(assets_selected, begin_date, df_currency_rate)

      df_assets = df_convert_type(pd.read_json(get_data('asset/')), df_currency_rate)

      our_portfolio_id = get_portfolio_id(df_assets, 'EPITA_PTF_10')
      our_portfolio = get_portfolio(our_portfolio_id)

      reset_assets_portfolio(our_portfolio_id, our_portfolio)#FIXME ATTENTIN A SUPRIMMER AVANT DE POST POUR AVOIR LES ACTIFS DANS LE PORTFOLIO

      ref_portfolio_id = get_portfolio_id(df_assets, 'REF')
      our_sharpe = compare_portfolios(our_portfolio_id, ref_portfolio_id, begin_date, end_date)
      if (max(max_sharpe[0], convert_float(our_sharpe)) > max_sharpe[0]):
        max_sharpe = (max(max_sharpe[0], convert_float(our_sharpe)), stock_percent[i])
    else:
      print("INVALID VALUE CONSTRAINTS NOT RESPECTED")
    print("-------------------------------------------------------------------------------------------------------")
  print("MAX SHARPE IS " + str(max_sharpe[0]) + "FOR VALUE " + str(max_sharpe[1]) + " IN GIVEN ARRAY")
main()

-------------------------------------------------------------------------------------------------------
STOCK PERCENT = 0.55
weights : [1.00000000e+00 1.25258621e+01 2.47423714e+01 3.22378662e+01
 3.72342502e+01 7.06410208e+01 8.62255163e+01 3.96578405e+02
 5.07467194e+02 6.12358854e+02 8.01737004e+02 8.32292513e+02
 8.48115684e+02 8.71219420e+02 1.81218139e+03 4.32080320e+03
 5.66243660e+03 6.19593691e+03 1.18618336e+05]
nav score max = 2.2014137357473373e-07
nav score min = -3.030436346307397e-09
assets stock score = 3.7206336855888367e-07
check nav constraint : True
check stock constraint : True
check minimal buy amount constraint : True
check declimalisation constraint : True
sharpe ratio in our portfolio : 2,720937848804
sharpe ratio in ref portfolio : -0,337925081462
-------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------
STOCK PE

In [31]:
our_portfolio_id = 1829
our_portfolio = get_portfolio(our_portfolio_id)
reset_assets_portfolio(our_portfolio_id, our_portfolio)#on reset le contenu du portfolio (plus safe)

In [32]:
def get_data2(endpointApi, start_date=None, end_date=None, full_response=False, columns=list()):
    payload = {'start_date': start_date, 'end_date': end_date, 'fullResponse': full_response}
    res = requests.get(URL + endpointApi, params=payload, auth=AUTH, verify=False)
    return res.content.decode('utf-8')

def post_data2(endpointApi, data=None, start_date=None, end_date=None, full_response=False, columns=list()):
    payload = {'start_date': start_date, 'end_date': end_date, 'fullResponse': full_response}
    res = requests.post(URL + endpointApi, data, params=payload, auth=AUTH, verify=False)
    return res.content.decode('utf-8')

def put_data2(endpointApi, data=None, start_date=None, end_date=None, full_response=False, columns=list()):
    payload = {'start_date': start_date, 'end_date': end_date, 'fullResponse': full_response}
    res = requests.put(URL + endpointApi, data, params=payload, auth=AUTH, verify=False)
    return res.content.decode('utf-8')

pd.options.display.max_seq_items = 2000

URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')

assets = [2024, 1958, 2165, 1971, 2023, 1882, 2194, 1485, 1538, 1877, 1612, 1566, 2034, 2062, 1910, 2196, 1872, 2066, 2195]
weights = [97479, 49588, 247, 108907, 14786371, 538644, 3909, 121, 226043, 705976, 9619, 2760, 772439, 63792, 100283, 4451, 105982, 76886, 8525]

#assets = [2024]
#weights = [100]

put = "{'label':'EPITA_PTF_10','currency':{'code':'EUR'},'type':'front','values':{'2016-06-01':["
for i in range(len(assets)):
    put += "{'asset':{'asset':" + str(assets[i]) + ",'quantity':" + str(weights[i]) + "}}"
    if (i != len(assets) -1):
        put += ","
put += "]}}"



tmp = "{'ratio'=[10, 12, 13],'asset'=[1829],'start_date'='2016-06-01','end_date'='2020-09-30','frequency':null}"
r = put_data2('portfolio/1829/dyn_amount_compo', put)
print('put {}'.format(r))

#print('REF')
#print(get_portfolio(2201))
#print('OUR')
#print(str(get_portfolio(1829)))
#print('GET {}'.format(get_data))
#df_portfolio = get_portfolio(portfolio_id)
#reset_assets_portfolio(portfolio_id, df_portfolio)
#put_portfolio(1829, get_portfolio(2201), ):
data = get_data('portfolio/{}/dyn_amount_compo'.format(2201))

print(data)
data = get_data('portfolio/{}/dyn_amount_compo'.format(1829))

print(data)
r = post_data2('ratio/invoke', tmp)
print(r)

put 
{"label":"REF","currency":{"code":"EUR"},"type":"front","values":{"2018-01-01":[{"asset":{"asset":1860,"quantity":6.0}},{"asset":{"asset":1900,"quantity":10.0}},{"currency":{"currency":{"code":"EUR"},"amount":300.0}}]}}
{"label":"EPITA_PTF_10","currency":{"code":"EUR"},"type":"front","values":{"2016-06-01":[{"asset":{"asset":1538,"quantity":226043.0}},{"asset":{"asset":1958,"quantity":49588.0}},{"asset":{"asset":2023,"quantity":1.4786371E7}},{"asset":{"asset":2024,"quantity":97479.0}},{"asset":{"asset":1612,"quantity":9619.0}},{"asset":{"asset":1485,"quantity":121.0}},{"asset":{"asset":2062,"quantity":63792.0}},{"asset":{"asset":1872,"quantity":105982.0}},{"asset":{"asset":2194,"quantity":3909.0}},{"asset":{"asset":2034,"quantity":772439.0}},{"asset":{"asset":2066,"quantity":76886.0}},{"asset":{"asset":1971,"quantity":108907.0}},{"asset":{"asset":2195,"quantity":8525.0}},{"asset":{"asset":2196,"quantity":4451.0}},{"asset":{"asset":2165,"quantity":247.0}},{"asset":{"asset":1877,"qu

In [39]:
begin_date = '2016-06-01'
end_date = '2020-09-30'
assets = get_df_asset(begin_date, df_currency_rate)
assets_id = [asset_id for asset_id in assets['ASSET_DATABASE_ID'].values]
assets_id

nav_values = []
nav = get_cotation_values(assets_id[0], begin_date, begin_date, df_currency_rate)['nav'].values[0]
nav_values.append(nav)
"""for asset_id in assets_id:
  nav = get_cotation_values(asset_id, begin_date, begin_date, df_currency_rate)['nav'].values[0]
  nav_values.append(nav)"""

nav_values

[36.851]

In [38]:
get_cotation_values(1845, begin_date, '2020-09-30', df_currency_rate)

,date,nav,gross,volume,high,real_close_price,feed_source,low,ask,asset,pl,bid,close,return,open
0,2016-06-01,36.851000000000,36.851000000000,143199.0,37.125,36.851000000000,-10.0,36.675,36.799,1845,0.037000000000,36.723,36.851000000000,0.001005052426,36.745
1,2016-06-02,36.900000000000,36.900000000000,147480.0,37.145,36.900000000000,-10.0,36.594,36.999,1845,0.049000000000,36.946,36.900000000000,0.001329678978,36.786
2,2016-06-03,36.052000000000,36.052000000000,190934.0,37.200,36.052000000000,-10.0,35.800,36.277,1845,-0.848000000000,36.219,36.052000000000,-0.022981029810,36.900
3,2016-06-06,36.540000000000,36.540000000000,82279.0,36.775,36.540000000000,-10.0,36.082,36.587,1845,0.488000000000,36.534,36.540000000000,0.013536003550,36.190
4,2016-06-07,37.246000000000,37.246000000000,119105.0,37.347,37.246000000000,-10.0,36.470,37.172,1845,0.706000000000,37.150,37.246000000000,0.019321291735,36.590
5,2016-06-08,37.127000000000,37.127000000000,84371.0,37.400,37.127000000000,-10.0,37.055,37.115,1845,-0.119000000000,37.078,37.127000000000,-0.003194973957,37.130
6,2016-06-09,36.263000000000,36.263000000000,107502.0,37.188,36.263000000000,-10.0,36.037,36.351,1845,-0.864000000000,36.259,36.263000000000,-0.023271473591,37.110
7,2016-06-10,35.369000000000,35.369000000000,146126.0,36.260,35.369000000000,-10.0,35.300,35.310,1845,-0.894000000000,35.287,35.369000000000,-0.024653227808,36.200
8,2016-06-13,34.907000000000,34.907000000000,244656.0,35.460,34.907000000000,-10.0,34.300,34.962,1845,-0.462000000000,34.904,34.907000000000,-0.013062286183,35.100
9,2016-06-14,33.800000000000,33.800000000000,206910.0,35.033,33.800000000000,-10.0,33.638,33.823,1845,-1.107000000000,33.699,33.800000000000,-0.031712836967,34.750


In [ ]:
data = get_data('asset/')
df = pd.read_json(data)
df_assets = df_convert_type(df, df_currency_rate)
df_assets